In [1]:
import torch
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import time

import datasets.mvtec as mvtec

from feature_extraction import Resnet18Features, WideResnet50Features, extractEmbeddingVectors, getIndices
# from distribution_fitting import getMeanCov
from score_calculation import calculatePatchScore, calculateImageScore, calculateImageClassification, calculatePatchClassification
# from padim_utils import saveFeatures, loadFeatures
# from __init__ import AnomalyDetector, imageAnomalyDetection


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
# device = torch.device('cpu')
device

device(type='cuda')

In [3]:
class_name = 'down'
data_path = '../data/aligned_pucks/'
dataset = mvtec.MVTecDataset(data_path, class_name=class_name, is_train=True)
dataloader = DataLoader(dataset, batch_size=32, pin_memory=True)
len(dataset)
# x = next(iter(dataloader))[0][0].unsqueeze(0)
x = next(iter(dataloader))[0][:20]
x.shape

torch.Size([20, 3, 224, 224])

In [4]:
features_model = Resnet18Features(device)
# features_model = WideResnet50Features(device)

In [5]:
tic = time.perf_counter()
mean = torch.load('./distributions/up_mean_test.pkl').to(device)
cov_inv = torch.load('./distributions/up_cov_inv_test.pkl').to(device)
print(time.perf_counter()-tic)


0.06584810500498861


In [6]:
indices = getIndices(100, 448, device)
embedding_vectors = extractEmbeddingVectors(features_model, x, device, indices=indices)

/home/plejd/anaconda3/envs/vision/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [7]:
tic = time.perf_counter()
patch_scores = calculatePatchScore(mean, cov_inv, embedding_vectors, device=device, do_gaussian_filter=True)
print(time.perf_counter()-tic)


0.26284912799019367


In [8]:
print(patch_scores.shape)
patch_scores[1]

torch.Size([20, 56, 56])


tensor([[39.0265, 38.7949, 38.3410,  ..., 38.9249, 39.5321, 39.8438],
        [38.8017, 38.5816, 38.1495,  ..., 38.8452, 39.4566, 39.7708],
        [38.3605, 38.1626, 37.7730,  ..., 38.6925, 39.3099, 39.6280],
        ...,
        [37.2169, 37.0863, 36.8220,  ..., 32.1420, 32.4493, 32.6096],
        [37.2117, 37.0850, 36.8290,  ..., 32.0424, 32.3241, 32.4706],
        [37.1995, 37.0754, 36.8244,  ..., 31.9896, 32.2583, 32.3977]],
       device='cuda:0')

In [9]:
patch_classifications = calculatePatchClassification(patch_scores, 10)
image_scores = calculateImageScore(patch_scores)
image_classifications = calculateImageClassification(image_scores, 10)
